In [21]:
import os

if not os.path.isfile("squadid_pred.csv"):
    !wget -O "squadid_pred.csv" "https://huggingface.co/muhammadravi251001/fine-tuned-DatasetQAS-Squad-ID-with-xlm-roberta-large-without-ITTL-without-freeze-LR-1e-05/resolve/main/results/output/output_df.csv"

if not os.path.isfile("tydiqaid_pred.csv"):
    !wget -O "tydiqaid_pred.csv" "https://huggingface.co/muhammadravi251001/fine-tuned-DatasetQAS-TYDI-QA-ID-with-xlm-roberta-large-without-ITTL-without-freeze-LR-1e-05/raw/main/results/output/output_df.csv"

if not os.path.isfile("idkmrc_pred.csv"):
    !wget -O "idkmrc_pred.csv" "https://huggingface.co/muhammadravi251001/fine-tuned-DatasetQAS-IDK-MRC-with-xlm-roberta-large-without-ITTL-without-freeze-LR-1e-05/raw/main/results/output/output_df.csv"

--2023-10-31 10:17:10--  https://huggingface.co/muhammadravi251001/fine-tuned-DatasetQAS-Squad-ID-with-xlm-roberta-large-without-ITTL-without-freeze-LR-1e-05/raw/main/results/output/output_df.csv
Resolving huggingface.co (huggingface.co)... 13.33.33.20, 13.33.33.110, 13.33.33.102, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133 [text/plain]
Saving to: ‘squadid_pred.csv’

squadid_pred.csv    100%[===================>]     133  --.-KB/s    in 0s      

2023-10-31 10:17:13 (26.4 MB/s) - ‘squadid_pred.csv’ saved [133/133]

--2023-10-31 10:17:14--  https://huggingface.co/muhammadravi251001/fine-tuned-DatasetQAS-TYDI-QA-ID-with-xlm-roberta-large-without-ITTL-without-freeze-LR-1e-05/raw/main/results/output/output_df.csv
Resolving huggingface.co (huggingface.co)... 13.33.33.20, 13.33.33.102, 13.33.33.110, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.20|:443... connected.
HTTP request sent,

In [57]:
import pandas as pd

squadid_df = pd.read_csv("squadid_pred.csv", index_col=0).drop(columns=['Answer Type', 'Reasoning Type']).fillna('')
tydiqaid_df = pd.read_csv("tydiqaid_pred.csv", index_col=0).drop(columns=['Answer Type', 'Reasoning Type']).fillna('')
idkmrc_df = pd.read_csv("idkmrc_pred.csv", index_col=0).drop(columns=['Answer Type', 'Reasoning Type']).fillna('')

In [93]:
def filtering_only_wrong_answer(data):
    return data[data['Prediction Answer'] != data['Gold Answer']].reset_index(drop=True)

In [94]:
wrong_squadid_df = filtering_only_wrong_answer(squadid_df)
wrong_tydiqaid_df = filtering_only_wrong_answer(tydiqaid_df)
wrong_idkmrc_df = filtering_only_wrong_answer(idkmrc_df)

# Analyse unanswerable vs answerable row

In [115]:
def counting_reversed_wrong_unanswerable(data, name):
    
    print(name.upper())
    print()
    
    count_empty_pred_and_non_empty_gold = ((data['Prediction Answer'] == '') & (data['Gold Answer'] != '')).sum()
    count_non_empty_pred_and_empty_gold = ((data['Prediction Answer'] != '') & (data['Gold Answer'] == '')).sum()
    
    total_wrong_data = len(data)
    
    placeholder1 = f"{round(count_empty_pred_and_non_empty_gold / total_wrong_data, 3) * 100}%"
    placeholder2 = f"{round(count_non_empty_pred_and_empty_gold / total_wrong_data, 3) * 100}%"
    
    print(f"Banyak data yang diprediksi UNANS tetapi gold-nya ANS: {count_empty_pred_and_non_empty_gold} ({placeholder1})")
    print(f"Banyak data yang diprediksi ANS tetapi gold-nya UNANS: {count_non_empty_pred_and_empty_gold} ({placeholder2})")
    print(f"Banyak keseluruhan data yang prediksinya berbeda dengan gold: {total_wrong_data}")
    
    print("=" * 70)

In [128]:
print("ANALYSE THIS\n")

counting_reversed_wrong_unanswerable(wrong_squadid_df, "squadid")
counting_reversed_wrong_unanswerable(wrong_tydiqaid_df, "tydiqaid")
counting_reversed_wrong_unanswerable(wrong_idkmrc_df, "idkmrc")

ANALYSE THIS

SQUADID

Banyak data yang diprediksi UNANS tetapi gold-nya ANS: 384 (6.4%)
Banyak data yang diprediksi ANS tetapi gold-nya UNANS: 966 (16.1%)
Banyak keseluruhan data yang prediksinya berbeda dengan gold: 5996
TYDIQAID

Banyak data yang diprediksi UNANS tetapi gold-nya ANS: 21 (7.3%)
Banyak data yang diprediksi ANS tetapi gold-nya UNANS: 1 (0.3%)
Banyak keseluruhan data yang prediksinya berbeda dengan gold: 287
IDKMRC

Banyak data yang diprediksi UNANS tetapi gold-nya ANS: 43 (23.0%)
Banyak data yang diprediksi ANS tetapi gold-nya UNANS: 32 (17.1%)
Banyak keseluruhan data yang prediksinya berbeda dengan gold: 187


# Analyse word vs sentence row

In [126]:
def counting_length_answer(data, name):
    
    print(name.upper())
    print()
    
    pred_answer_length = data['Prediction Answer'].str.split().apply(len)
    gold_answer_length = data['Gold Answer'].str.split().apply(len)
    
    count_word_pred_and_sentence_gold = ((pred_answer_length == 1) & (gold_answer_length > 1)).sum()
    count_sentence_pred_and_word_gold = ((pred_answer_length > 1) & (gold_answer_length == 1)).sum()
    
    total_wrong_data = len(data)
    
    placeholder1 = f"{round(count_word_pred_and_sentence_gold / total_wrong_data, 3) * 100}%"
    placeholder2 = f"{round(count_sentence_pred_and_word_gold / total_wrong_data, 3) * 100}%"
    
    print(f"Banyak data yang diprediksi WORD tetapi gold-nya SENTENCE: {count_word_pred_and_sentence_gold} ({placeholder1})")
    print(f"Banyak data yang diprediksi SENTENCE tetapi gold-nya WORD: {count_sentence_pred_and_word_gold} ({placeholder2})")
    print(f"Banyak keseluruhan data yang prediksinya berbeda dengan gold: {total_wrong_data}")
    
    print("=" * 70)

In [129]:
print("ANALYSE THIS\n")

counting_length_answer(wrong_squadid_df, "squadid")
counting_length_answer(wrong_tydiqaid_df, "tydiqaid")
counting_length_answer(wrong_idkmrc_df, "idkmrc")

ANALYSE THIS

SQUADID

Banyak data yang diprediksi WORD tetapi gold-nya SENTENCE: 505 (8.4%)
Banyak data yang diprediksi SENTENCE tetapi gold-nya WORD: 817 (13.600000000000001%)
Banyak keseluruhan data yang prediksinya berbeda dengan gold: 5996
TYDIQAID

Banyak data yang diprediksi WORD tetapi gold-nya SENTENCE: 24 (8.4%)
Banyak data yang diprediksi SENTENCE tetapi gold-nya WORD: 44 (15.299999999999999%)
Banyak keseluruhan data yang prediksinya berbeda dengan gold: 287
IDKMRC

Banyak data yang diprediksi WORD tetapi gold-nya SENTENCE: 6 (3.2%)
Banyak data yang diprediksi SENTENCE tetapi gold-nya WORD: 18 (9.6%)
Banyak keseluruhan data yang prediksinya berbeda dengan gold: 187


# Analyse overlapping answer

In [140]:
def count_overlapping(row):
    pred_words = set(row['Prediction Answer'].split())
    gold_words = set(row['Gold Answer'].split())
    return 1 if len(pred_words.intersection(gold_words)) > 0 else 0

def counting_overlapping_answer(data, name):
    
    print(name.upper())
    print()
    
    overlap_count = data.apply(count_overlapping, axis=1).sum()
    
    total_wrong_data = len(data)
    
    placeholder1 = f"{round(overlap_count / total_wrong_data, 3) * 100}%"
    
    print(f"Banyak data yang diprediksi overlap dengan gold: {overlap_count} ({placeholder1})")
    print(f"Banyak keseluruhan data yang prediksinya berbeda dengan gold: {total_wrong_data}")
    
    print("=" * 70)

In [141]:
print("ANALYSE THIS\n")

counting_overlapping_answer(wrong_squadid_df, "squadid")
counting_overlapping_answer(wrong_tydiqaid_df, "tydiqaid")
counting_overlapping_answer(wrong_idkmrc_df, "idkmrc")

ANALYSE THIS

SQUADID

Banyak data yang diprediksi overlap dengan gold: 3354 (55.900000000000006%)
Banyak keseluruhan data yang prediksinya berbeda dengan gold: 5996
TYDIQAID

Banyak data yang diprediksi overlap dengan gold: 182 (63.4%)
Banyak keseluruhan data yang prediksinya berbeda dengan gold: 287
IDKMRC

Banyak data yang diprediksi overlap dengan gold: 92 (49.2%)
Banyak keseluruhan data yang prediksinya berbeda dengan gold: 187


In [99]:
def sampling_data(data, sample_num=100):
    return data.sample(n=sample_num).reset_index(drop=True)

In [100]:
sampled_squadid_df = sampling_data(wrong_squadid_df)
sampled_tydiqaid_df = sampling_data(wrong_tydiqaid_df)
sampled_idkmrc_df = sampling_data(wrong_idkmrc_df)